<font size="5">OECD CO2 emissions into Trino pipeline</font>

Load Environment Variables

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [2]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'mdt_sandbox'
ingest_table = 'sf_oecd_imgr_fco2'

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [5]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [6]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('sandbox',)


Load CO2 emissions file (updated from https://stats.oecd.org/Index.aspx?DataSetCode=IO_GHG_2021)

In [7]:
import pandas as pd
import csv
import ParseXLS as parser


oecd_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/OECD/IMGR_FCO2.csv')
oecd_file.download_file(f'/tmp/IMGR_FCO2.csv')
#df = parser.process('OECD_IMGR_FCO2.ini','OECD.csv') 
df =pd.read_csv(f'/tmp/IMGR_FCO2.csv')
df= df[['Indicator','COU','Country','PAR','IND','Industry','TIME','Value']]
df=df.convert_dtypes()
dict = {'Indicator': 'attribute',
        'COU': 'country_iso_code',
        'Country': 'country_name',
        'PAR': 'partner_iso_code',
        'IND': 'industry_code',
        'Industry' : 'industry_name',
        'TIME'   : 'validity_date',
        'Value' : 'value' }
 
# call rename () method
df.rename(columns=dict,
          inplace=True)


df['value_units'] = 'Mt CO2'



df
         


        






,attribute,country_iso_code,country_name,partner_iso_code,industry_code,industry_name,validity_date,value,value_units
0,Foreign CO2 emissions embodied in gross imports,OECD,OECD member countries,WLD,DTOTAL,TOTAL,1995,1518.407,Mt CO2
1,Foreign CO2 emissions embodied in gross imports,OECD,OECD member countries,WLD,DTOTAL,TOTAL,1996,1447.641,Mt CO2
2,Foreign CO2 emissions embodied in gross imports,OECD,OECD member countries,WLD,DTOTAL,TOTAL,1997,1434.138,Mt CO2
3,Foreign CO2 emissions embodied in gross imports,OECD,OECD member countries,WLD,DTOTAL,TOTAL,1998,1620.643,Mt CO2
4,Foreign CO2 emissions embodied in gross imports,OECD,OECD member countries,WLD,DTOTAL,TOTAL,1999,1797.149,Mt CO2
...,...,...,...,...,...,...,...,...,...
334651,Foreign CO2 emissions embodied in gross imports,ZOTH,Other regions,MMR,D35,"Electricity, gas, steam and air conditioning s...",2014,0.0,Mt CO2
334652,Foreign CO2 emissions embodied in gross imports,ZOTH,Other regions,MMR,D35,"Electricity, gas, steam and air conditioning s...",2015,0.0,Mt CO2
334653,Foreign CO2 emissions embodied in gross imports,ZOTH,Other regions,MMR,D35,"Electricity, gas, steam and air conditioning s...",2016,0.0,Mt CO2
334654,Foreign CO2 emissions embodied in gross imports,ZOTH,Other regions,MMR,D35,"Electricity, gas, steam and air conditioning s...",2017,0.0,Mt CO2


In [8]:
import pycountry
df_country = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df_country = df_country["alpha_3"].rename({'alpha_3':'country_iso_code'})
df_country
                                           
                                                      


0      ABW
1      AFG
2      AGO
3      AIA
4      ALA
      ... 
244    WSM
245    YEM
246    ZAF
247    ZMB
248    ZWE
Name: alpha_3, Length: 249, dtype: object

In [9]:
import osc_ingest_trino as osc
df = df.convert_dtypes()
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())


drop table if exists osc_datacommons_dev.mdt_sandbox.sf_oecd_imgr_fco2

[(True,)]


In [10]:
df.info(verbose=True)
        
        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334656 entries, 0 to 334655
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   attribute         334656 non-null  string 
 1   country_iso_code  334656 non-null  string 
 2   country_name      334656 non-null  string 
 3   partner_iso_code  334656 non-null  string 
 4   industry_code     334656 non-null  string 
 5   industry_name     334656 non-null  string 
 6   validity_date     334656 non-null  Int64  
 7   value             334656 non-null  Float64
 8   value_units       334656 non-null  string 
dtypes: Float64(1), Int64(1), string(7)
memory usage: 23.6 MB


In [11]:
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 5000, verbose = True))

constructed fully qualified table name as: "mdt_sandbox.sf_oecd_imgr_fco2"
inserting 5000 records
  ('Foreign CO2 emissions embodied in gross imports', 'OECD', 'OECD member countries', 'WLD', 'DTOTAL', 'TOTAL', 1995, 1518.407, 'Mt CO2')
  ('Foreign CO2 emissions embodied in gross imports', 'OECD', 'OECD member countries', 'WLD', 'DTOTAL', 'TOTAL', 1996, 1447.641, 'Mt CO2')
  ('Foreign CO2 emissions embodied in gross imports', 'OECD', 'OECD member countries', 'WLD', 'DTOTAL', 'TOTAL', 1997, 1434.138, 'Mt CO2')
  ...
  ('Foreign CO2 emissions embodied in gross imports', 'AUS', 'Australia', 'NONOECD', 'DTOTAL', 'TOTAL', 2002, 44.56, 'Mt CO2')
batch insert result: [(5000,)]
inserting 5000 records
  ('Foreign CO2 emissions embodied in gross imports', 'AUS', 'Australia', 'NONOECD', 'DTOTAL', 'TOTAL', 2003, 56.854, 'Mt CO2')
  ('Foreign CO2 emissions embodied in gross imports', 'AUS', 'Australia', 'NONOECD', 'DTOTAL', 'TOTAL', 2004, 69.185, 'Mt CO2')
  ('Foreign CO2 emissions embodied in gros

In [12]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}" + "where validity_date=2007"""
pd.read_sql(sql, engine)

,attribute,country_iso_code,country_name,partner_iso_code,industry_code,industry_name,validity_date,value,value_units
0,Foreign CO2 emissions embodied in gross imports,BGR,Bulgaria,ROW,DTOTAL,TOTAL,2007,2.067,Mt CO2
1,Foreign CO2 emissions embodied in gross imports,BGR,Bulgaria,APEC,DTOTAL,TOTAL,2007,8.527,Mt CO2
2,Foreign CO2 emissions embodied in gross imports,BGR,Bulgaria,ASEAN,DTOTAL,TOTAL,2007,0.231,Mt CO2
3,Foreign CO2 emissions embodied in gross imports,BGR,Bulgaria,EASIA,DTOTAL,TOTAL,2007,2.268,Mt CO2
4,Foreign CO2 emissions embodied in gross imports,BGR,Bulgaria,EU27_2020,DTOTAL,TOTAL,2007,6.522,Mt CO2
...,...,...,...,...,...,...,...,...,...
13939,Foreign CO2 emissions embodied in gross imports,IRL,Ireland,MAR,DTOTAL,TOTAL,2007,0.114,Mt CO2
13940,Foreign CO2 emissions embodied in gross imports,IRL,Ireland,PER,DTOTAL,TOTAL,2007,0.004,Mt CO2
13941,Foreign CO2 emissions embodied in gross imports,IRL,Ireland,PHL,DTOTAL,TOTAL,2007,0.070,Mt CO2
13942,Foreign CO2 emissions embodied in gross imports,IRL,Ireland,ROU,DTOTAL,TOTAL,2007,0.051,Mt CO2
